In [1]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

# set kinetic parameters
with open("labs.json") as infile:
    jsdata = json.load(infile)

params = jsdata["cal1"]

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )

params["nTime"] = 50
params["relaxTime"] = 3
params["mixTime"] = 20
params["roomT"] = random.gauss(298,2)
params["slope"] = (2*random.random()-1) / params["nTime"]
params["error"] = 0.5

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Calorimetry #1

Same as the lab, after the heat capacity of the calorimeter has been measured.

In [3]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

def measure(mA,mB,CalCp,SaltCp):

    molesA = mA * params["molAcid"] / 1000
    molesB = mB * params["molBase"] / 1000
    molesFormed = min(molesA,molesB)
    negHeat = params["deltaHf"] * molesFormed
    
    totalMass = mA + mB
    heatCapacitySolution = totalMass * SaltCp
    
    deltaT =  negHeat / (heatCapacitySolution + CalCp)
    
    return deltaT
    
def calc(btn):
    out_P.clear_output()
    
    # Measurement result
    deltaT = measure(
        float(massA.value),
        float(massB.value),
        float(CalCp.value),
        float(SaltCp.value))
    
    res = pd.DataFrame(columns=["Time [min]" , "T [K]"])

    for i in range(0, params["nTime"]):
        var_list = []
        var_list.append(i)
        tempOut = params["roomT"] + random.gauss(0,params["error"]) + params["slope"] * i
        if (i > params["mixTime"]):
            tempOut = tempOut + deltaT * (1-math.exp(-(i-params["mixTime"])/params["relaxTime"] ))
            
        var_list.append(tempOut)
        res.loc[len(res)] = var_list
    
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(respath):
        os.remove(respath)
    
    with out_P:
        out_P.clear_output()

# interactive buttons ---
btn_calc = ipw.Button(description="Perform Experiment", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

btn_reset = ipw.Button(description="Reset Experiment", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

# ---
reset(btn_reset)

# --- create the boxes and sliders
rows = []

CalCp = ipw.Text("1000")
rows.append(ipw.HBox([ipw.Label('Heat capacity of calorimeter  :  '),CalCp]))

SaltCp = ipw.Text("0.1")
rows.append(ipw.HBox([ipw.Label('Specific heat capacity of the salt  :  '),SaltCp]))

massA = ipw.FloatLogSlider(value=1, min=-4, max=3)
massB = ipw.FloatLogSlider(value=1, min=-4, max=3)
rows.append(ipw.HBox([ipw.Label(value="Mass of acid solution [g]", layout=ipw.Layout(width="250px")), massA]))
rows.append(ipw.HBox([ipw.Label(value="Mass of base solution [g]", layout=ipw.Layout(width="250px")), massB]))

rows.append(ipw.HBox([btn_reset]))
rows.append(ipw.HBox([btn_calc]))
rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)